In [8]:
import pandas as pd
import numpy as np
from scipy import sparse
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor,RandomTreesEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV


def root_mean_squared_error_plain(y_true, y_pred):
        return abs(np.sqrt(np.mean((y_pred - y_true)**2))) 
    
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
def strd(s):
    return (s-np.mean(s))/np.std(s)

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


In [22]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

## Test Improvement

In [12]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

hasher = RandomTreesEmbedding(n_estimators=1000, random_state=0, max_depth=3)
x_train = hasher.fit_transform(x_train)
x_test = hasher.transform(x_test)

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error_plain(y_pred, y_true))


1.6270168950989707


In [57]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

hasher = RandomTreesEmbedding(n_estimators=100, random_state=0, max_depth=3)
x_train = hasher.fit_transform(x_train)
x_test = hasher.transform(x_test)

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

xtrainpred = strd(clf.predict(x_train).flatten().reshape(-1,1))
xtestpred = strd(clf.predict(x_test).flatten().reshape(-1,1))
x_train = sparse.hstack((x_train,xtrainpred)).toarray()
x_test = sparse.hstack((x_test,xtestpred)).toarray()

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs,
        shuffle='batch')

Train on 3567 samples, validate on 892 samples
Epoch 1/1000
3567/3567 [==============================] - 0s 128us/step - loss: 21.0235 - val_loss: 9.5810
Epoch 2/1000
3567/3567 [==============================] - 0s 15us/step - loss: 6.5592 - val_loss: 7.8780
Epoch 3/1000
3567/3567 [==============================] - 0s 15us/step - loss: 4.6174 - val_loss: 5.7099
Epoch 4/1000
3567/3567 [==============================] - 0s 15us/step - loss: 4.1300 - val_loss: 6.4028
Epoch 5/1000
3567/3567 [==============================] - 0s 15us/step - loss: 4.0080 - val_loss: 5.8185
Epoch 6/1000
3567/3567 [==============================] - 0s 15us/step - loss: 3.6432 - val_loss: 5.3880
Epoch 7/1000
3567/3567 [==============================] - 0s 15us/step - loss: 3.0766 - val_loss: 5.0455
Epoch 8/1000
3567/3567 [==============================] - 0s 15us/step - loss: 2.7525 - val_loss: 4.2692
Epoch 9/1000
3567/3567 [==============================] - 0s 15us/step - loss: 2.5199 - val_loss: 4.3015
Epoch 

Epoch 78/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4716 - val_loss: 1.4333
Epoch 79/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4664 - val_loss: 1.4333
Epoch 80/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4811 - val_loss: 1.4335
Epoch 81/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4817 - val_loss: 1.4333
Epoch 82/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4738 - val_loss: 1.4334
Epoch 83/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4740 - val_loss: 1.4333
Epoch 84/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4710 - val_loss: 1.4332
Epoch 85/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.4703 - val_loss: 1.4332
Epoch 86/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.4664 - val_loss: 1.4332
Epoch 87/1000
3567/3567 [=====================

## NN+ElNet - Actual

In [59]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

hasher = RandomTreesEmbedding(n_estimators=100, random_state=0, max_depth=3)
x_train = hasher.fit_transform(x_train)
x_test = hasher.transform(x_test)

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

xtrainpred = strd(clf.predict(x_train).flatten().reshape(-1,1))
xtestpred = strd(clf.predict(x_test).flatten().reshape(-1,1))
x_train = sparse.hstack((x_train,xtrainpred)).toarray()
x_test = sparse.hstack((x_test,xtestpred)).toarray()

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 79us/step - loss: 17.0552
Epoch 2/1000
4459/4459 [==============================] - 0s 13us/step - loss: 5.2076
Epoch 3/1000
4459/4459 [==============================] - 0s 12us/step - loss: 4.2908
Epoch 4/1000
4459/4459 [==============================] - 0s 13us/step - loss: 3.6296
Epoch 5/1000
4459/4459 [==============================] - 0s 13us/step - loss: 2.9556
Epoch 6/1000
4459/4459 [==============================] - 0s 13us/step - loss: 2.6680
Epoch 7/1000
4459/4459 [==============================] - 0s 12us/step - loss: 2.3312
Epoch 8/1000
4459/4459 [==============================] - 0s 12us/step - loss: 2.1631
Epoch 9/1000
4459/4459 [==============================] - 0s 13us/step - loss: 2.0503
Epoch 10/1000
4459/4459 [==============================] - 0s 12us/step - loss: 2.0452
Epoch 11/1000
4459/4459 [==============================] - 0s 12us/step - loss: 1.9904
Epoch 12/1000
4459/4459 [==========================

4459/4459 [==============================] - 0s 13us/step - loss: 1.4348
Epoch 96/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4351
Epoch 97/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4347
Epoch 98/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4350
Epoch 99/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4344
Epoch 100/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4344
Epoch 101/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4351
Epoch 102/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4351
Epoch 103/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4342
Epoch 104/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4348
Epoch 105/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.4346
Epoch 106/1000
4459/4459 [=========================

In [27]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_boost.csv',index=False)

49342/49342 [==============================] - 1s 16us/step
          ID      target
0  000137c73  1752246.00
1  00021489f  2149595.00
2  0004d7953  2535024.25
3  00056a333  2129491.00
4  00056d8eb  2149595.00
